In [26]:
# %pip install nuscenes-devkit

In [27]:
# %pip install "numpy<2"

In [28]:
# %pip install plotly

In [29]:
import os
import json
import math

import torch

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import mmcv
from mmdet3d.apis import init_model, inference_detector

import cv2
from PIL import Image

from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import RadarPointCloud
from nuscenes.eval.detection.evaluate import NuScenesEval
from nuscenes.eval.detection.data_classes import DetectionConfig
from nuscenes.prediction import PredictHelper
from nuscenes.utils.splits import create_splits_scenes

# import open3d as o3d
# from open3d.j_visualizer import JVisualizer

In [30]:
%matplotlib inline

## Load Model

In [ ]:
# Add data loader commands

In [31]:
cfg_file = "pointpillars_hv_secfpn_sbn-all_8xb4-2x_nus-3d.py"

In [32]:
ckpt_file = "hv_pointpillars_secfpn_sbn-all_4x8_2x_nus-3d_20210826_225857-f19d00a3.pth"

In [33]:
model = init_model(cfg_file, ckpt_file, device='cuda:0')

Loads checkpoint by local backend from path: hv_pointpillars_secfpn_sbn-all_4x8_2x_nus-3d_20210826_225857-f19d00a3.pth


/home/012392471@SJSUAD/master_project/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(
/home/012392471@SJSUAD/anaconda3/envs/project/lib/python3.10/site-packages/mmengine/runner/checkpoint.py:347: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_global

## Load Dataset

In [34]:
# DATAROOT = '/home/012392471@SJSUAD/master_project/data/nuscenes'

In [35]:
DATAROOT = '/home/012392471@SJSUAD/master_project/data/nuscenes_mini'

In [36]:
# nusc = NuScenes(version='v1.0-trainval', dataroot=DATAROOT, verbose=True)
nusc = NuScenes(version='v1.0-mini', dataroot=DATAROOT, verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.346 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [37]:
helper = PredictHelper(nusc)

In [38]:
nusc.list_scenes()

scene-0061, Parked truck, construction, intersectio... [18-07-24 03:28:47]   19s, singapore-onenorth, #anns:4622
scene-0103, Many peds right, wait for turning car, ... [18-08-01 19:26:43]   19s, boston-seaport, #anns:2046
scene-0655, Parking lot, parked cars, jaywalker, be... [18-08-27 15:51:32]   20s, boston-seaport, #anns:2332
scene-0553, Wait at intersection, bicycle, large tr... [18-08-28 20:48:16]   20s, boston-seaport, #anns:1950
scene-0757, Arrive at busy intersection, bus, wait ... [18-08-30 19:25:08]   20s, boston-seaport, #anns:592
scene-0796, Scooter, peds on sidewalk, bus, cars, t... [18-10-02 02:52:24]   20s, singapore-queensto, #anns:708
scene-0916, Parking lot, bicycle rack, parked bicyc... [18-10-08 07:37:13]   20s, singapore-queensto, #anns:2387
scene-1077, Night, big street, bus stop, high speed... [18-11-21 11:39:27]   20s, singapore-hollandv, #anns:890
scene-1094, Night, after rain, many peds, PMD, ped ... [18-11-21 11:47:27]   19s, singapore-hollandv, #anns:1762
sc

In [51]:
scene_splits = create_splits_scenes()

In [40]:
mini_train_scenes = scene_splits['mini_train']
mini_train_scenes

['scene-0061',
 'scene-0553',
 'scene-0655',
 'scene-0757',
 'scene-0796',
 'scene-1077',
 'scene-1094',
 'scene-1100']

## Visualization

In [41]:
# test_scene = nusc.scene[0]
# first_sample_token = test_scene['first_sample_token']
# my_sample = nusc.get('sample', first_sample_token)
# lidar_top_data = nusc.get('sample_data', my_sample['data']['LIDAR_TOP'])

In [42]:
# bin_path = os.path.join(nusc.dataroot, lidar_top_data['filename'])

In [43]:
# points = np.fromfile(bin_path, dtype=np.float32)
# points = points.reshape(-1, 4) # (x, y, z, depth)
# points = points[:, :-1]

In [44]:
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(points)

# # Visualize using WebVisualizer
# o3d.visualization.draw([pcd])

## Helpers

In [45]:
def yaw_to_quaternion(yaw):
    # Calculate half of the yaw angle
    half_yaw = yaw * 0.5
    
    # Compute the quaternion components
    w = math.cos(half_yaw)
    x = 0.0
    y = 0.0
    z = math.sin(half_yaw)
    
    return [w, x, y, z]

## Constants

In [46]:
class_names = [
    'car',
    'truck',
    'trailer',
    'bus',
    'construction_vehicle',
    'bicycle',
    'motorcycle',
    'pedestrian',
    'traffic_cone',
    'barrier',
]

In [47]:
sensor = 'LIDAR_TOP'

## Perform Predictions

In [48]:
prediction_results = {
    "meta": {
        "use_camera": False,
        "use_lidar": True,
        "use_radar": False,
        "use_map": False, 
        "use_external": False 
    },
    "results": {}
}

In [49]:
# START_INDEX = 0
# END_INDEX = 3

In [56]:
# for i in range(START_INDEX, END_INDEX):
#     test_scene = nusc.scene[i]
for scene_name in mini_train_scenes:
    test_scene = nusc.get('scene', scene_name)
    first_sample_token = test_scene['first_sample_token']
    my_sample = nusc.get('sample', first_sample_token)
    lidar_top_data = nusc.get('sample_data', my_sample['data'][sensor])

    bin_path = os.path.join(nusc.dataroot, lidar_top_data['filename'])
    results = inference_detector(model, bin_path)[0]

    dict_key = f"{first_sample_token}"
    arr = []
    
    n = len(results.pred_instances_3d.bboxes_3d)

    for index in range(n):
        box = results.pred_instances_3d.bboxes_3d[index]
        score = results.pred_instances_3d.scores_3d[index]
        label = results.pred_instances_3d.labels_3d[index]
        detection_name = class_names[label]

        box = box.tensor[0]
        box = box.cpu().numpy().tolist()

        score = score.cpu().numpy().tolist()
        
        prediction_object = {
            'sample_token': first_sample_token,
            'translation': box[0:3],  # Center of 3D box
            'size': box[3:6],  # Dimensions of 3D box
            'rotation': yaw_to_quaternion(box[6]),  # Rotation as quaternion
            'velocity': box[7:],  # vx, vy
            'detection_name': detection_name,  # e.g., 'car', 'pedestrian'
            'detection_score': score,  # Between 0 and 1
            'attribute_name': ""
        }

        arr.append(prediction_object)

    prediction_results["results"][dict_key] = arr

scene-0061


KeyError: 'scene-0061'

In [ ]:
with open('results.json', 'w') as fp:
    json.dump(prediction_results, fp)

## Perform Evaluation

In [ ]:
with open('detection_cvpr_2019.json', 'r') as fp:
    config_str = json.load(fp)

In [ ]:
# dot notation doesn't work on dictionaries, only objects
detection_config = DetectionConfig(
    class_range = config_str["class_range"],
    dist_fcn = config_str["dist_fcn"],
    dist_ths = config_str["dist_ths"],
    dist_th_tp = config_str["dist_th_tp"],
    min_recall = config_str["min_recall"],
    min_precision = config_str["min_precision"],
    max_boxes_per_sample = config_str["max_boxes_per_sample"],
    mean_ap_weight = config_str["mean_ap_weight"]
)

In [ ]:
evaluator = NuScenesEval(nusc,
                         config=detection_config,
                         eval_set='mini_val',
                         result_path="results.json",
                         output_dir='outputs/')